In [1]:
import pandas as pd
import os
from matplotlib import pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
fin = "/h/dsmith/physionet_data/meanfill/physionet_data.parquet"
df = pd.read_parquet(fin, engine="pyarrow")
npts = 10
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,SepsisEver,id
0,83.656309,97.170036,36.841922,123.387789,82.676393,64.263019,18.527826,33.017979,-0.441724,24.382587,...,207.257752,61.00,0.0,1.000000,0.000000,-7.80,1.0,0.0,0.0,37337
1,63.000000,95.000000,36.900000,93.000000,71.000000,49.000000,18.527826,33.017979,-0.441724,24.382587,...,207.257752,61.00,0.0,1.000000,0.000000,-7.80,2.0,0.0,0.0,37337
2,64.000000,91.000000,36.900000,115.000000,60.000000,39.000000,20.000000,33.017979,-0.441724,24.382587,...,207.257752,61.00,0.0,1.000000,0.000000,-7.80,3.0,0.0,0.0,37337
3,71.000000,92.000000,36.900000,117.000000,60.000000,41.000000,18.000000,33.017979,-0.441724,24.382587,...,207.257752,61.00,0.0,1.000000,0.000000,-7.80,4.0,0.0,0.0,37337
4,71.000000,92.000000,36.900000,117.000000,60.000000,41.000000,18.000000,33.017979,-0.441724,24.382587,...,207.257752,61.00,0.0,1.000000,0.000000,-7.80,5.0,0.0,0.0,37337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545247,94.000000,97.000000,37.060000,180.000000,119.000000,76.000000,20.000000,33.017979,0.000000,21.000000,...,107.000000,67.77,0.0,0.492043,0.507957,-27.09,53.0,0.0,0.0,20336
1545248,89.000000,98.000000,37.060000,168.000000,107.000000,72.000000,26.000000,33.017979,0.000000,21.000000,...,107.000000,67.77,0.0,0.492043,0.507957,-27.09,54.0,0.0,0.0,20336
1545249,85.000000,99.000000,37.060000,163.000000,106.000000,68.000000,14.000000,33.017979,0.000000,21.000000,...,107.000000,67.77,0.0,0.492043,0.507957,-27.09,55.0,0.0,0.0,20336
1545250,87.000000,99.000000,36.220000,163.000000,107.000000,69.000000,13.000000,33.017979,0.000000,21.000000,...,107.000000,67.77,0.0,0.492043,0.507957,-27.09,56.0,0.0,0.0,20336


In [4]:
badcols = ["ICULOS", "SepsisLabel", "SepsisEver", "index"]
slim = df[[i for i in df.columns if i not in badcols]]
variations = slim.groupby("id").std().mean()
static_vars = variations[variations == 0].index.tolist()
puny_vars = variations[(0 < variations) & (variations < 0.05)].index.tolist()
moving_vars = [i for i in variations.index.tolist() if i not in puny_vars + static_vars]

In [5]:
data_groups = {
    "earlymean": slim[moving_vars + ["id"]].groupby("id").head(npts).groupby("id").mean().reset_index(),
    "earlystd": slim[moving_vars + ["id"]].groupby("id").head(npts).groupby("id").std().reset_index(),
    "labels": df.groupby("id")["SepsisEver"].tail(1).astype(int),
    "skew": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").skew().reset_index(),
    "std": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").std().reset_index(),
    "mean": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").mean().reset_index(),
    "static": slim[static_vars + ["id"]].groupby("id").tail(1).reset_index()
}

In [6]:
var_sets = {
    "late": ["std", "mean", "static"],
    "early": ["earlymean", "earlystd", "static"],
    "static": ["static"],
    "moving": ["mean", "std", "skew"],
    "tail": ["mean"],
    "kitchen_sink": ["std", "mean", "earlymean", "earlystd", "static", "skew"],
}
        
def assemble_data(var_set):
    chosen_sets = var_sets[var_set]

    dataset = data_groups[chosen_sets[0]].copy()
    dataset.columns = [
        (i + "_" + chosen_sets[0]).replace(f"id_{chosen_sets[0]}", "id")
        for i in dataset.columns
    ]
    
    for i in range(1, len(chosen_sets)):
        dataset = dataset.merge(
            data_groups[chosen_sets[i]], on="id", suffixes=("", "_" + chosen_sets[i])
        )
    
    del dataset["id"]
    for i in dataset.columns:
        if "index" in i:
            del dataset[i]
        
    return dataset

assemble_data("kitchen_sink")

,HR_std,O2Sat_std,Temp_std,SBP_std,MAP_std,DBP_std,Resp_std,EtCO2_std,BaseExcess_std,HCO3_std,...,Phosphate_skew,Potassium_skew,Bilirubin_total_skew,TroponinI_skew,Hct_skew,Hgb_skew,PTT_skew,WBC_skew,Fibrinogen_skew,Platelets_skew
0,3.055505,1.106797,0.267135,20.117726,9.267073,4.477102,2.006240,0.0,0.421637,0.421637,...,1.778781,0.000000,0.0,0.0,0.000000,0.000000,1.778781,0.000000e+00,0.0,0.000000
1,3.852849,1.950071,0.321498,7.761908,6.290204,4.412419,3.162717,0.0,0.000000,0.632456,...,0.000000,3.162278,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000
2,16.593506,2.945807,0.249366,0.000000,9.186947,0.000000,2.044641,0.0,0.000000,1.581139,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-4.937859e-15,0.0,0.000000
3,4.655045,0.761942,0.458650,13.301629,7.211873,5.337498,3.093003,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000
4,3.180234,0.421637,0.385292,21.638700,15.926240,0.000000,5.807084,0.0,0.000000,0.000000,...,0.000000,1.035098,0.0,0.0,1.035098,1.035098,0.000000,-1.035098e+00,0.0,1.035098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39546,1.334375,0.632456,0.306232,2.586826,1.862644,1.960725,2.309401,0.0,0.000000,0.000000,...,0.000000,0.929587,0.0,0.0,1.778781,0.000000,0.000000,1.778781e+00,0.0,1.778781
39547,8.032933,1.025057,0.165164,8.107765,8.197859,7.366638,2.338159,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000
39548,3.541814,1.932184,0.303681,7.400450,5.021067,4.346135,4.143268,0.0,0.000000,0.000000,...,0.000000,-1.778781,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000
39549,4.195235,0.527046,0.134990,27.548745,14.704497,8.778762,1.779513,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000


In [7]:
# from sklearn.ensemble import RandomForestClassifier as RFC
# from sklearn.model_selection import cross_validate

# class BoostModel(object):
    
#     def __init__(self, X, y, group="none", nparts=10):
#         self.Xdata = X
#         self.ydata = y
#         self.nparts = nparts
#         self.results = pd.DataFrame()
#         self.dgroup = group
        
#     def optimize(self):
#         depths = [2, 10, 20]
#         min_leafs = [2, 32, 128]
        
#         for i in depths:
#             for j in min_leafs:
#                 model = RFC(max_depth=i, min_samples_leaf=j, class_weight="balanced")
#                 cv_results = cross_validate(model, self.Xdata, self.ydata, cv=self.nparts, scoring=("f1", "accuracy"))
#                 self.results = self.results.append(
#                     {
#                         "depth": i,
#                         "min_leafs": j,
#                         "data_group": self.dgroup,
#                         "test_acc": cv_results["test_accuracy"].mean(),
#                         "test_acc_std": cv_results["test_accuracy"].std(),
#                         "test_f1": cv_results["test_accuracy"].mean(),
#                         "test_f1_std": cv_results["test_accuracy"].std(),
#                     },
#                     ignore_index=True
#                 )
#                 print(
#                     "Random Forest with depth {} and min_leafs {} got accuracy = {} +/- {} with f1 = {} on {}.".format(
#                         i, j, 
#                         cv_results["test_accuracy"].mean(), cv_results["test_accuracy"].std(),
#                         cv_results["test_f1"]
#                         self.dgroup
#                     )
#                 )
                
#     def get_optimized(self, param="f1"):
#         try:
#             assert(len(self.results.index) > 0)
#         except AssertionError:
#             raise ValueError("BoostModel.get_optimized requires that optimize be run first")
        
#         optparams = self.results.sort_values(by=f"test_{param}").iloc[0]
#         optimized = RFC(
#             max_depth=int(optparams["depth"]),
#             min_samples_leaf=int(optparams["min_leafs"]),
#             class_weight="balanced"
#         )
#         optimized.fit(self.Xdata, self.ydata)
        
#         return optimized

from src import BoostModel

opt_models = []
columns = []
all_results = pd.DataFrame()
for i in var_sets:
    print("\nPreparing {}...".format(i))
    xdata = assemble_data(i)
    print("Data prepped. Running models.")
    BM = BoostModel(xdata, data_groups["labels"], group=i)
    BM.optimize()
    all_results = all_results.append(BM.results, ignore_index=True)
    all_results.to_csv("model_opt_results_mean.csv", index=False)
    all_results = all_results.append(BM.results, ignore_index=True)
    opt_models.append(BM.get_optimized())
    columns.append(xdata.columns)


Preparing late...
Data prepped. Running models.
Random Forest with depth 2 and min_leafs 2 got accuracy = 0.6389439907247739 +/- 0.12361977003275809 with f1 = [0.10414658 0.09613869 0.09356725 0.0861244  0.11184211 0.09090909
 0.11070111 0.10548885 0.11645102 0.10882487] on late.
Random Forest with depth 2 and min_leafs 32 got accuracy = 0.595379816412906 +/- 0.13878559913761623 with f1 = [0.092827   0.09943182 0.09505703 0.11832061 0.099631   0.1111683
 0.11480757 0.10185585 0.09758657 0.10276339] on late.
Random Forest with depth 2 and min_leafs 128 got accuracy = 0.5931054111024046 +/- 0.15614464851405674 with f1 = [0.1011396  0.09082484 0.09936575 0.10928014 0.11954766 0.10246856
 0.11085973 0.11751412 0.11590204 0.11425577] on late.
Random Forest with depth 10 and min_leafs 2 got accuracy = 0.8914558372182503 +/- 0.03372957036273571 with f1 = [0.02515723 0.02105263 0.01311475 0.03846154 0.03625378 0.03992016
 0.0862069  0.05414552 0.08787346 0.06656101] on late.
Random Forest wit

Random Forest with depth 20 and min_leafs 128 got accuracy = 0.9130738630625886 +/- 0.014890462227904805 with f1 = [0.01398601 0.00724638 0.01408451 0.02777778 0.02352941 0.0273224
 0.04932735 0.02777778 0.03373494 0.0626506 ] on moving.

Preparing tail...
Data prepped. Running models.
Random Forest with depth 2 and min_leafs 2 got accuracy = 0.6368969153737893 +/- 0.0619135285023738 with f1 = [0.10447761 0.10505319 0.09497965 0.10237924 0.11703511 0.10562972
 0.10834132 0.0998004  0.10810811 0.10506389] on tail.
Random Forest with depth 2 and min_leafs 32 got accuracy = 0.6167185117199434 +/- 0.0666873341202916 with f1 = [0.07931404 0.10350584 0.08644689 0.11226358 0.10789324 0.1007584
 0.1038526  0.10067873 0.11240506 0.10583581] on tail.
Random Forest with depth 2 and min_leafs 128 got accuracy = 0.6133347032272827 +/- 0.06827472774770897 with f1 = [0.10820244 0.10863095 0.09566517 0.10727406 0.10885167 0.0982659
 0.09635723 0.10189702 0.110668   0.09837468] on tail.
Random Forest w

In [8]:
def showNfeats(model, columns, n=15):
    df = pd.DataFrame({
        "feature": columns,
        "importances": model.feature_importances_,
    }).sort_values(by="importances", ascending=False).head(n)
    print(df)
    return df
    
for i, j in list(zip(opt_models, columns)):
    print(i)
    showNfeats(i, j)

RandomForestClassifier(class_weight='balanced', max_depth=2,
                       min_samples_leaf=128)
          feature  importances
36            MAP     0.057101
51        Glucose     0.041896
5         DBP_std     0.041167
63      Platelets     0.040024
34           Temp     0.038261
3         SBP_std     0.035183
50     Creatinine     0.034155
29        WBC_std     0.032187
68    HospAdmTime     0.032108
48        Calcium     0.031545
55      Potassium     0.030814
25  TroponinI_std     0.027565
17   Chloride_std     0.025085
26        Hct_std     0.024418
58            Hct     0.023799
RandomForestClassifier(class_weight='balanced', max_depth=2, min_samples_leaf=2)
                   feature  importances
48                 Calcium     0.043719
45                     AST     0.043324
51                 Glucose     0.040627
17      Chloride_earlymean     0.037284
68             HospAdmTime     0.035665
32                      HR     0.027886
11         PaCO2_earlymean     0.0277

In [9]:
late_data = assemble_data("late")
late_model = RFC(max_depth=10, min_samples_leaf=5, class_weight="balanced")
late_model.fit(late_data, data_groups["labels"])

early_data = assemble_data("early")
early_model = RFC(max_depth=10, min_samples_leaf=5, class_weight="balanced")
early_model.fit(early_data, data_groups["labels"])

NameError: name 'RFC' is not defined

In [ ]:
import numpy as np

def get_features(model, data):
    features = pd.DataFrame(
        data=np.asarray([data.columns.tolist(), model.feature_importances_.tolist()]).T, 
        columns=["feature", "importance"]
    )
    return features.sort_values(by="importance", ascending=False).reset_index(drop=True)

print("late:")
print(get_features(late_model, late_data).head(20))

print("\nearly:")
print(get_features(early_model, early_data).head(20))

In [ ]:
xdata = assemble_data("late")
print("Data prepped. Running models.")
BM = BoostModel(xdata, data_groups["labels"], group="late")
BM.optimize()
all_results = all_results.append(BM.results, ignore_index=True)
all_results.to_csv("model_opt_results.csv", index=False)